In [ ]:
from dataclasses import dataclass
import yaml
import tweepy
import sys
import asyncio

from tweepy.asynchronous import AsyncClient

: 

In [ ]:
@dataclass(init=True, repr=True, frozen=True)
class TwitterUser:
    
    id_ : str
    username: str
    name: str
    tweets: str

In [ ]:
@dataclass(init=True, repr=True, frozen=True)
class Tweet:

    id_: str
    text: str
    context_annotations: list

In [ ]:
class TwitterData:

    def __init__(self, filename: str = "twitter-oauth.yaml") -> None:

        temp_twitter_auth_data = {}

        with open(filename, "r") as yamlfile:
            temp_twitter_auth_data = yaml.load(
                yamlfile, Loader=yaml.FullLoader)

        self.twitter_auth_creds = temp_twitter_auth_data['TWITTER']

        self.client = AsyncClient(access_token=self.twitter_auth_creds['ACCESS_TOKEN'],
                                access_token_secret=self.twitter_auth_creds['ACCESS_TOKEN_SECRET'],
                                consumer_key=self.twitter_auth_creds['COSUMER_KEY'],
                                consumer_secret=self.twitter_auth_creds['CONSUMER_SECRET'],
                                bearer_token=self.twitter_auth_creds['BEARER_TOKEN'],
                                wait_on_rate_limit=True)

    async def get_user_data_from_username(self, screen_name: str):

        print(f'now fetching for: {screen_name}')

        user_data = await self.client.get_user(username=screen_name, user_fields=['public_metrics'])

        # return await self.get_twitter_user_from_user_data(user_data)

        user_id = user_data.data.id

        recent_tweets = self.get_users_recent_tweets(user_id)

        return TwitterUser(
            user_id,
            user_data.data.username,
            user_data.data.name,
            await recent_tweets
        )

    async def get_users_recent_tweets(self, user_id, max_tweets=100):

        tweets = await self.client.get_users_tweets(
            user_id, tweet_fields=['context_annotations'], max_results=max_tweets)

        final_tweets = []

        if not tweets.data:
            return []

        for tweet in tweets.data:

            tweet_context_annotations = []

            tweet_id = tweet.id
            tweet_text = tweet.text
            tweet_temp_context_annotations = tweet.context_annotations

            for context in tweet_temp_context_annotations:

                entity_name = context['entity']['name']

                if not entity_name in tweet_context_annotations:
                    tweet_context_annotations.append(entity_name)

            final_tweet = Tweet(
                tweet_id,
                tweet_text,
                tweet_context_annotations
            )

            final_tweets.append(final_tweet)

        return final_tweets

In [ ]:
def transform_tweets_to_text(tweets: list):
    
    text = ""
    
    for tweet in tweets:
        
        text+=tweet.text + " "
        
        for entity in tweet.context_annotation:
            text+= entity.strip() + " "
            
    return text